In [1]:
import collections
import itertools
import pyld
import json
import phenopackets as pps
import pyhpo
import jsonpath_ng
from google.protobuf.json_format import MessageToJson
from google.protobuf.timestamp_pb2 import Timestamp
import glob
import gzip
import csv
import statistics as stat
import logging

In [2]:
logger = logging.getLogger(__name__)

JSON-LD, being very RDF-y, will represent 0..* fields as single items if there's just one, and lists if more than one. Sometimes it's more convenient to be consistent.

In [3]:
def ensure_list(value):
    if isinstance(value, list):
        return value
    return [value]

A dictionary of MONDO ids to labels (should probably use ClinGen-preferred when available)

In [5]:
with gzip.open('MONDO.csv.gz', 'rt') as csvf:
    '''Keys will be like "MONDO_0044647", values like "kyphosis-lateral tongue atrophy-myofibrillar myopathy syndrome"'''
    reader = csv.DictReader(csvf)
    class_prefix = 'http://purl.obolibrary.org/obo/'
    mondo_lookup = dict((row['Class ID'][len(class_prefix):], row['Preferred Label']) for row in reader if row['Class ID'].startswith(class_prefix))

load the HPO ontology using pyhpo

In [7]:
pyhpo.Ontology()
print(pyhpo.Ontology.version())

2025-01-16
The Ontology has been built before already


We will need this later for protobuf representation

In [8]:
timestamp = Timestamp()

In [9]:
j = json.load(open('input/gene-validity-202512/gg_02212618-0725-43db-9a4c-7929faeb878av1.0.json'))

In [10]:
j['subject']['disease']

'obo:MONDO_0013135'

The idea behind using JSON-LD framing is two-fold:
* It gives us a quick way to get down to the probands
* By specifying `embed` of `@always`, we should be able to avoid having to keep our own dictionary of objects (*for the most part*... pyld does not appear to handle re-embeddeding `dc:source` when needed below)

In [11]:
frame = {'@context': j['@context'], '@type': 'Proband', '@embed': '@always'}

In [12]:
framed = pyld.jsonld.frame(j, frame)

In [13]:
len(framed['@graph'])

10

In [14]:
framed['@graph'][0].keys()

dict_keys(['id', 'type', 'dc:source', 'rdfs:label', 'ageType', 'ageUnit', 'ageValue', 'allele', 'detectionMethod', 'firstTestingMethod', 'phenotypeFreeText', 'previousTestingDescription', 'secondTestingMethod', 'sex', 'variant'])

*FIXME*: not really sure what to use for `resources`, so just going with what is in the phenopacket store for now...

In [15]:
resources = [
  pps.Resource(**resource) for resource in [
    {
      "id": "geno",
      "name": "Genotype Ontology",
      "url": "http://purl.obolibrary.org/obo/geno.owl",
      "version": "2022-03-05",
      "namespace_prefix": "GENO",
      "iri_prefix": "http://purl.obolibrary.org/obo/GENO_"
    },
    {
      "id": "hgnc",
      "name": "HUGO Gene Nomenclature Committee",
      "url": "https://www.genenames.org",
      "version": "06/01/23",
      "namespace_prefix": "HGNC",
      "iri_prefix": "https://www.genenames.org/data/gene-symbol-report/#!/hgnc_id/"
    },
    {
      "id": "omim",
      "name": "An Online Catalog of Human Genes and Genetic Disorders",
      "url": "https://www.omim.org",
      "version": "January 4, 2023",
      "namespace_prefix": "OMIM",
      "iri_prefix": "https://www.omim.org/entry/"
    },
    {
      "id": "so",
      "name": "Sequence types and features ontology",
      "url": "http://purl.obolibrary.org/obo/so.obo",
      "version": "2021-11-22",
      "namespace_prefix": "SO",
      "iri_prefix": "http://purl.obolibrary.org/obo/SO_"
    },
    {
      "id": "hp",
      "name": "human phenotype ontology",
      "url": "http://purl.obolibrary.org/obo/hp.owl",
      "version": "2024-07-01",
      "namespace_prefix": "HP",
      "iri_prefix": "http://purl.obolibrary.org/obo/HP_"
    }
  ]
]

In [16]:
def phenotype_element_from_id(hpoid, **kwargs):
    try:
        term = pyhpo.Ontology.get_hpo_object(hpoid)
    except:
        logger.error(f'unable to find term for {hpoid}')
    return pps.PhenotypicFeature(type=pps.OntologyClass(id=term.id, label=term.name), **kwargs)

In [17]:
def obo_to_labeled_phenotype(obo):
    try:
        term = pyhpo.Ontology.get_hpo_object(obo.replace('obo:HP_', 'HP:'))
        return {'id': term.id, 'label': term.name}
    except:
        return {'id': obo}

dcsource_path = jsonpath_ng.parse('$.."dc:source"')

def id_or_value(x):
    if isinstance(x, dict):
        return x.get('id', '')
    return x

def find_sources(proband):
    try:
        dc_sources = [proband['dc:source']]
    except KeyError:
        try:
          found = reversed(dcsource_path.find(proband))
          # Note: there is some hacky handling of non-embeded dc:source by testing
          #   for a string and ignoring (since it *should* be embedded elsewhere, then
          #   but this will probably break when there are multiple probands from some
          #   source in same file. Maybe pyld's @embed: @always will work better
          #   some day...
          sources_by_id = dict((x.value.get('id', 'UNKNOWN'), x.value)
                               for x in found
                               if not isinstance(x.value, str))
          dc_sources = list(sources_by_id.values())
        except Exception as e:
            print(f'ERROR finding dc:source with {proband=}')
            print(e)
            dc_sources = []
    return dc_sources

def proband_phenotype_report(proband):
    phenotypes = [obo_to_labeled_phenotype(p) for p in ensure_list(proband.get('phenotypes', []))]
    sources = find_sources(proband)
    #FIXME
    try:
        pmids = ','.join(set(s['id'].replace('https://pubmed.ncbi.nlm.nih.gov/', '') for s in sources))
    except:
        pmids = ''
    return {
        'id': proband['id'],
        'pmids': pmids,
        'label': proband['rdfs:label'],
        'phenotypes': phenotypes,
    }

proband_phenotype_report(framed['@graph'][4])

{'id': 'https://genegraph.clinicalgenome.org/r/5d76c7de-d884-409c-858d-a9c7995aa5cf',
 'pmids': '',
 'label': '1875',
 'phenotypes': [{'id': 'HP:0004313',
   'label': 'Decreased circulating antibody concentration'}]}

In [18]:
framed['@graph'][4]

{'id': 'https://genegraph.clinicalgenome.org/r/5d76c7de-d884-409c-858d-a9c7995aa5cf',
 'type': 'Proband',
 'dc:source': 'https://pubmed.ncbi.nlm.nih.gov/19804848',
 'rdfs:label': '1875',
 'ageType': 'AgeAtOnset',
 'ageUnit': 'Months',
 'ageValue': 4,
 'allele': {'id': 'https://genegraph.clinicalgenome.org/r/ce6508ac-b270-4f95-a36a-a356ecdb286f',
  'type': 'https://terms.ga4gh.org/VariationDescriptor',
  'http://www.w3.org/2004/02/skos/core#prefLabel': 'NM_006949.4(STXBP2):c.1430C>T (p.Pro477Leu)',
  'https://terms.ga4gh.org/CanonicalReference': {'id': 'http://reg.genome.network/allele/CA254261'}},
 'detectionMethod': 'Whole-genome homozygosity mapping. STXBP2 was then PCR amplified and there was Sanger sequencing of coding exons and adjacent intronic sequences.',
 'firstTestingMethod': 'Homozygosity mapping',
 'phenotypeFreeText': '5/8 HLH criteria met',
 'phenotypes': 'obo:HP_0004313',
 'previousTestingDescription': 'Screened for mutations in PRF1, UNC13D, and STX11.',
 'secondTesting

In [19]:
find_sources(framed['@graph'][0])

['https://pubmed.ncbi.nlm.nih.gov/19804848']

In [20]:
def proband_to_phenopacket(proband):
    references = []
    for dc_source in find_sources(proband):
        # until/unless genegraph provides dc:source as an object again instead of just an id/reference...
        #pmid = dc_source['id'].replace('https://pubmed.ncbi.nlm.nih.gov/', '')
        #reference = pps.ExternalReference(id=f'PMID:{pmid}', reference=dc_source['id'], description=dc_source['dc:title'])
        pmid = dc_source.replace('https://pubmed.ncbi.nlm.nih.gov/', '')
        reference = pps.ExternalReference(id=f'PMID:{pmid}', reference=dc_source)
        referenceEvidence = pps.Evidence(reference=reference, evidence_code=pps.OntologyClass(id="ECO:0006017", label="author statement from published clinical study used in manual assertion"))
        references.append(reference)
    phenotypes = [phenotype_element_from_id(p.replace('obo:HP_', 'HP:'), evidence=[referenceEvidence]) for p in ensure_list(proband.get('phenotypes', []))]
    metadata = pps.MetaData(external_references=references,
                            created=timestamp.GetCurrentTime(),
                            created_by="Automated import from ClinGen GCI data",
                            phenopacket_schema_version="2.0",
                            resources=resources)
    subject_args = {'id': f'{pmid}:{proband['rdfs:label']}'}
    try:
        subject_args['sex'] = pps.Sex.Value(proband['sex'])
    except:
        subject_args['sex'] = pps.Sex.UNKNOWN_SEX
    individual = pps.Individual(**subject_args)
    phenopacket = pps.Phenopacket(id=proband['id'], subject=individual, phenotypic_features=phenotypes, meta_data=metadata)
    return phenopacket

print(MessageToJson(proband_to_phenopacket(framed['@graph'][4])))

{
  "id": "https://genegraph.clinicalgenome.org/r/5d76c7de-d884-409c-858d-a9c7995aa5cf",
  "subject": {
    "id": "19804848:1875"
  },
  "phenotypicFeatures": [
    {
      "type": {
        "id": "HP:0004313",
        "label": "Decreased circulating antibody concentration"
      },
      "evidence": [
        {
          "evidenceCode": {
            "id": "ECO:0006017",
            "label": "author statement from published clinical study used in manual assertion"
          },
          "reference": {
            "id": "PMID:19804848",
            "reference": "https://pubmed.ncbi.nlm.nih.gov/19804848"
          }
        }
      ]
    }
  ],
  "metaData": {
    "createdBy": "Automated import from ClinGen GCI data",
    "resources": [
      {
        "id": "geno",
        "name": "Genotype Ontology",
        "url": "http://purl.obolibrary.org/obo/geno.owl",
        "version": "2022-03-05",
        "namespacePrefix": "GENO",
        "iriPrefix": "http://purl.obolibrary.org/obo/GENO_"
 

In [21]:
inputs = sorted(glob.glob('input/gene-validity-202512/*.json'))[::-1]
len(inputs)

3387

In [ ]:
proband_summaries = {}
for f in inputs:
    with open(f) as jf:
        j = json.load(jf)
    framed = pyld.jsonld.frame(j, frame)
    for proband in framed.get('@graph', []):
        if proband['id'] in proband_summaries:
            print(f'WARNING: more than one record for {proband['id']}, not sure which one to use!')
        summary = proband_phenotype_report(proband)
        summary['evidenceStrength'] = j.get('evidenceStrength', '')
        disease_ids = [x.replace('obo:', '').replace(':', '_') for x in ensure_list(j.get('subject', {}).get('disease', 'unknown'))]
        summary['disease'] = [{'id': did, 'label': mondo_lookup.get(did, '')} for did in disease_ids]
        proband_summaries[proband['id']] = summary


In [ ]:
probands_with_phenotypes = [ps for ps in proband_summaries.values() if len(ps['phenotypes'])> 0]
print(f'Probands with phenotypes: {len(probands_with_phenotypes)}')
print(f'Probands with phenotypes where evidence at least moderate: {len([x for x in probands_with_phenotypes if x.get('evidenceStrength') in ('Moderate', 'Strong', 'Definitive')])}')

In [ ]:
collections.Counter([x.get('disease', [{}])[0].get('label', '') for x in probands_with_phenotypes])

In [ ]:
with open('probands_with_phenotypes.tsv', 'wt') as ouf:
    print('\t'.join(('evidence_strength', 'evidence_id', 'pmid', 'label', 'phenotypes', 'phenotype_labels', 'diseases', 'disease_labels')), file=ouf)
    for line in probands_with_phenotypes:
        phenotypes = '|'.join(x['id'] for x in line.get('phenotypes'))
        phenotype_labels = '|'.join(x.get('label', '') for x in line.get('phenotypes'))
        diseases = '|'.join(x['id'] for x in line.get('disease'))
        disease_labels = '|'.join(x['label'] for x in line.get('disease'))
        print('\t'.join((
            line['evidenceStrength'],
            line['id'],
            line['pmids'],
            line['label'],
            phenotypes,
            phenotype_labels,
            diseases,
            disease_labels,
        )), file=ouf)

In [ ]:
phenotype_counts = [len(p['phenotypes']) for p in proband_summaries.values()]

In [ ]:
countdata = phenotype_counts
print(f'Among all {len(countdata)} probands:')
print(f'mean: {stat.mean(countdata)}')
print(f'stdev: {stat.stdev(countdata)}')
print(f'median: {stat.median(countdata)}')
print(f'max: {max(countdata)}')

In [ ]:
countdata = [pc for pc in phenotype_counts if pc != 0]
print(f'Among {len(countdata)} probands with at least one phenotype reported:')
print(f'mean: {stat.mean(countdata)}')
print(f'stdev: {stat.stdev(countdata)}')
print(f'median: {stat.median(countdata)}')
print(f'max: {max(countdata)}')

In [ ]:
terms_by_count = collections.Counter()
for p in proband_summaries.values():
    terms_by_count.update(pheno.get('label', pheno.get('id', 'UNKNOWN')) for pheno in p['phenotypes'])
terms_by_count